In [1]:
def mysql_to_pandas(table_name):
    #sql to pandas using sqlchemy
    import pandas as pd
    from sqlalchemy import create_engine
    from sqlalchemy.ext.declarative import declarative_base
    from sqlalchemy.orm import scoped_session, sessionmaker

    # db settings
    dbuser = 'root'
    dbpass = 'lnmiit'
    dbhost = 'localhost'
    dbname = 'btpdb'

    engine = create_engine("mysql://%s:%s@%s/%s?charset=utf8&use_unicode=0"
                           %(dbuser, dbpass, dbhost, dbname),
                           echo=False,
                           pool_recycle=1800)
    conn = engine.connect()
    df = pd.read_sql('select * from %s'%table_name, con=conn)  
    conn.close()
    return df

In [56]:
df_ama = mysql_to_pandas("amazon_AppleiPhone5sSilver16GB")
df_snap = mysql_to_pandas("snapdeal_appleiphone5s16gb")
print df_ama.shape,df_snap.shape

(3891, 5) (2054, 3)


In [57]:
def filtering(df):
    ind_with_keywords = df.review.str.\
    contains('warranty|amazon|snapdel|delivery|shipping|service|transporting|packing|package',\
    flags=re.IGNORECASE, regex=True, na=False)
    return df[ind_with_keywords]

In [78]:
df_ama_filtered = filtering(df_ama)
df_snap_filtered = filtering(df_snap)

In [79]:
print df_ama_filtered.shape,df_snap_filtered.shape

(1114, 5) (615, 3)


In [80]:
df_ama_filtered.head(2)

,Id,title,rating,upvotes,review
0,1,Good Phone. Got India 1 year warrenty - verified.,5,34,iPhone 5s was first released 2 years ago in No...
1,2,IS IT STILL WORTH IT IN 2016? [READ TO FIND OU...,4,67,FIRSTLY AMAZON DELIVERED THE PRODUCT WELL ON T...


In [81]:
df_snap_filtered.head(2)

,title,rating,review
4,Awesome Product with Super fast Delivery.,5,"Excellent packing, Product original, No need t..."
11,OSM Service,4,Product is delivered on time; gud service prov...


In [82]:
import numpy as np
df_ama_filtered['label'] = np.nan
df_snap_filtered['label'] = np.nan

/home/master/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/master/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [83]:
df_ama_filtered.loc[df_ama_filtered.rating>3,'label'] = 1
df_ama_filtered.loc[df_ama_filtered.rating<=3,'label'] = 0
df_snap_filtered.loc[df_snap_filtered.rating>3,'label'] = 1
df_snap_filtered.loc[df_snap_filtered.rating<=3,'label'] = 0

In [84]:
df_ama_filtered.head(2)

,Id,title,rating,upvotes,review,label
0,1,Good Phone. Got India 1 year warrenty - verified.,5,34,iPhone 5s was first released 2 years ago in No...,1
1,2,IS IT STILL WORTH IT IN 2016? [READ TO FIND OU...,4,67,FIRSTLY AMAZON DELIVERED THE PRODUCT WELL ON T...,1


In [85]:
df_snap_filtered.head(2)

,title,rating,review,label
4,Awesome Product with Super fast Delivery.,5,"Excellent packing, Product original, No need t...",1
11,OSM Service,4,Product is delivered on time; gud service prov...,1


In [93]:
print df_ama_filtered.loc[df_ama_filtered.label==1].shape,df_ama_filtered.loc[df_ama_filtered.label==0].shape
print df_snap_filtered.loc[df_snap_filtered.label==1].shape,df_snap_filtered.loc[df_snap_filtered.label==0].shape

(920, 6) (194, 6)
(602, 4) (13, 4)


In [94]:
df1 = df_ama_filtered
df2 = df_snap_filtered

In [103]:
df_labeled = pd.concat([df1[['review','label']], df2[['review','label']]])

In [104]:
print df1.shape,df2.shape,df_labeled.shape

(1114, 6) (615, 4) (1729, 2)


In [105]:
df_labeled.head()

,review,label
0,iPhone 5s was first released 2 years ago in No...,1
1,FIRSTLY AMAZON DELIVERED THE PRODUCT WELL ON T...,1
2,In the past 4 years I have owned Android and (...,1
3,iPhone 5s was first released 2 years ago in No...,1
5,I had returned the i phone on the very fourth ...,0


In [107]:
print df_labeled.loc[df_labeled.label==1].shape,df_labeled.loc[df_labeled.label==0].shape

(1522, 2) (207, 2)


In [108]:
df_labeled.to_csv( "./labeled_data.csv", index=False)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(y_true, y_pred)